# Usage of Opensearch Output Connector with Event Objects


The following example demonstrates the delivery of events to the opensearch output connector

In [10]:
%%bash
docker compose -f ../../../../../examples/compose/docker-compose.yml down -v  
docker compose -f ../../../../../examples/compose/docker-compose.yml up -d opensearch dashboards

 Network compose_opensearch  Creating
 Network compose_opensearch  Created
 Volume "compose_opensearch-data"  Creating
 Volume "compose_opensearch-data"  Created
 Container opensearch  Creating
 Container opensearch  Created
 Container dashboards  Creating
 Container dashboards  Created
 Container opensearch  Starting
 Container opensearch  Started
 Container dashboards  Starting
 Container dashboards  Started


In [26]:
from typing import Iterator
from logprep.factory import Factory
from logprep.util.time import TimeParser
from logprep.ng.connector.opensearch.output import OpensearchOutput
from logprep.ng.event.log_event import LogEvent
from logprep.ng.event.event_state import EventStateType

config = {
    "type": "ng_opensearch_output",
    "hosts": [
        "127.0.0.1:9200"
    ],
    "default_index": "processed",
    "default_op_type": "create",
    "message_backlog_size": 1000,
    "timeout": 10000,
    "flush_timeout": 60,
    "user": "admin",
    "secret": "admin",
    "desired_cluster_status": ["green", "yellow"]
}

opensearch_output: OpensearchOutput = Factory.create({"my-opensearch": config})
opensearch_output.setup()


events: Iterator = [
    LogEvent({"message": f"Event {i}", "@timestamp": TimeParser.now()}, original=b"", state=EventStateType.PROCESSED)
    for i in range(2000)
]

# store events in the Opensearch output
for event in events:
    opensearch_output.store(event)

# Flush the output to ensure all events are sent
opensearch_output.flush()

# Verify that all events are delivered
for event in events:
    assert event.state == EventStateType.DELIVERED, f"Event {event.data['message']} not delivered | State: {event.state}"
    



The following case demonstrates error handling in the opensearch output connector.
It depends on the first example. There the `@timestamp` field was indexed by opensearch and we defined it as a time field.
Next we try to send other data to this field to provoke a `mapper parsing error`.

In [28]:
events: Iterator = [
    LogEvent({"message": f"Event {i}", "@timestamp": "this is not a timestamp"}, original=b"", state=EventStateType.PROCESSED)
    for i in range(2000)
]

# store events in the Opensearch output
for event in events:
    opensearch_output.store(event)

# Flush the output to ensure all events are sent
opensearch_output.flush()
# Verify that all events are delivered
for event in events:
    assert event.state == EventStateType.FAILED
    assert len(event.errors) == 1
    print (f"Event {event.data['message']} failed with error: {event.errors[0]}")

Event Event 0 failed with error: BulkError: {'type': 'mapper_parsing_exception', 'reason': "failed to parse field [@timestamp] of type [date] in document with id 'hWfcT5gBBm02-UH_EwM-'. Preview of field's value: 'this is not a timestamp'", 'caused_by': {'type': 'illegal_argument_exception', 'reason': 'failed to parse date field [this is not a timestamp] with format [strict_date_optional_time||epoch_millis]', 'caused_by': {'type': 'date_time_parse_exception', 'reason': 'Failed to parse with all enclosed parsers'}}}, status_code: 400, exception: None
Event Event 1 failed with error: BulkError: {'type': 'mapper_parsing_exception', 'reason': "failed to parse field [@timestamp] of type [date] in document with id 'hmfcT5gBBm02-UH_EwM-'. Preview of field's value: 'this is not a timestamp'", 'caused_by': {'type': 'illegal_argument_exception', 'reason': 'failed to parse date field [this is not a timestamp] with format [strict_date_optional_time||epoch_millis]', 'caused_by': {'type': 'date_time_